# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [2]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
  """
  Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
  it for the linear classifier. These are the same steps as we used for the
  SVM, but condensed to a single function.  
  """
  # Load the raw CIFAR-10 data
  cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
  # subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]
  mask = np.random.choice(num_training, num_dev, replace=False)
  X_dev = X_train[mask]
  y_dev = y_train[mask]
  
  # Preprocessing: reshape the image data into rows
  X_train = np.reshape(X_train, (X_train.shape[0], -1))
  X_val = np.reshape(X_val, (X_val.shape[0], -1))
  X_test = np.reshape(X_test, (X_test.shape[0], -1))
  X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
  
  # Normalize the data: subtract the mean image
  mean_image = np.mean(X_train, axis = 0)
  X_train -= mean_image
  X_val -= mean_image
  X_test -= mean_image
  X_dev -= mean_image
  
  # add bias dimension and transform into columns
  X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
  X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
  X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
  X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
  
  return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', y_test.shape
print 'dev data shape: ', X_dev.shape
print 'dev labels shape: ', y_dev.shape

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [4]:
import numpy as np
from random import shuffle

def softmax_loss_naive(W, X, y, reg):
  """
  Softmax loss function, naive implementation (with loops)
  Inputs:
  - W: D x c array of weights
  - X: N x D array of data. Data are D-dimensional columns
  - y: 1-dimensional array of length N with labels 0...C-1, for C classes
  - reg: (float) regularization strength
  Returns:
  a tuple of:
  - loss as single float
  - gradient with respect to weights W, an array of same size as W
  """
  # Initialize the loss and gradient to zero.
  data_loss = 0.0
  dW = np.zeros_like(W)
  n_sample, n_feature  = X.shape
  assert n_feature == W.shape[0]
  for i in xrange(n_sample):
    scores = np.dot(X[i],W)
    scores -= np.max(scores)
    scores = np.exp(scores)
    probability = scores / np.sum(scores)

    #accumulate the data_loss
    sample_loss = -np.log(probability[y[i]])
    data_loss += sample_loss / n_sample

    #accumulate the gradient with respect to every single data sample
    probability[y[i]] -= 1
    dW_single_sample = np.outer(X[i],probability) / n_sample
    dW += dW_single_sample

  reg_loss = 0.5 * reg * np.sum(W[:-1,:] ** 2)
  loss = data_loss + reg_loss

  return loss, dW

def softmax_loss_vectorized(W, X, y, reg):
  """
  Softmax loss function, vectorized version.
  Inputs and outputs are the same as softmax_loss_naive.
  """
  # Initialize the loss and gradient to zero.
  dW = np.zeros_like(W)
  n_sample = X.shape[0]

  assert X.shape[1] == W.shape[0]
  scores = np.dot(X,W)

  scores -= np.max(scores, axis = 1, keepdims = True)
  scores = np.exp(scores)
  probs = scores / np.sum(scores, axis = 1, keepdims = True)
  kl_loss = np.sum(-np.log(probs[range(n_sample),y])) / n_sample
  reg_loss = 0.5 * reg * np.sum(W[:-1,:] ** 2)
  loss = kl_loss + reg_loss

  dscores = probs
  dscores[range(n_sample),y] -= 1
  dscores /= n_sample
  dW = np.dot(X.T, dscores)

  return loss, dW

In [5]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

#from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print 'loss: %f' % loss
print 'sanity check: %f' % (-np.log(0.1))

loss: 2.338973
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** There are 10 classes in total. After normalization, the probability of the correct class is about 10%, which is 0.1.


In [6]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 1e2)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 1e2)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 4.144013 analytic: 4.144013, relative error: 1.830995e-08
numerical: 1.761216 analytic: 1.761216, relative error: 1.672674e-08
numerical: 0.080451 analytic: 0.080451, relative error: 1.533439e-07
numerical: 0.145145 analytic: 0.145145, relative error: 1.463204e-07
numerical: 1.268452 analytic: 1.268451, relative error: 2.772334e-08
numerical: -0.060458 analytic: -0.060458, relative error: 4.667356e-07
numerical: 0.101639 analytic: 0.101639, relative error: 1.498645e-07
numerical: -0.695755 analytic: -0.695755, relative error: 7.948496e-09
numerical: -1.809145 analytic: -1.809145, relative error: 2.499556e-08
numerical: -0.179706 analytic: -0.179706, relative error: 4.333396e-07
numerical: 0.443660 analytic: 0.435165, relative error: 9.666078e-03
numerical: -0.128323 analytic: -0.137962, relative error: 3.619774e-02
numerical: 0.375964 analytic: 0.365794, relative error: 1.371018e-02
numerical: 0.983938 analytic: 0.978784, relative error: 2.626056e-03
numerical: 0.053598 anal

In [7]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'naive loss: %e computed in %fs' % (loss_naive, toc - tic)

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic)

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print 'Loss difference: %f' % np.abs(loss_naive - loss_vectorized)
print 'Gradient difference: %f' % grad_difference

naive loss: 2.338973e+00 computed in 0.199303s
vectorized loss: 2.338973e+00 computed in 0.036854s
Loss difference: 0.000000
Gradient difference: 0.000000


In [21]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [5e4, 1e8]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
params = [(x,y) for x in learning_rates for y in regularization_strengths]

for (lr, reg) in params:
    print(lr, reg)
    softmax = Softmax()
    softmax.train(X_train,y_train,learning_rate = lr,reg = reg,num_iters = 1500,batch_size = 200,verbose = False)

    train_y_predict = softmax.predict(X_train)
    valid_y_predict = softmax.predict(X_val)
    train_accuracy = np.mean(y_train == train_y_predict)
    val_accuracy = np.mean(y_val == valid_y_predict)
    results[(lr,reg)] = (train_accuracy,val_accuracy)
    if val_accuracy > best_val:
        best_val = val_accuracy
        best_softmax = softmax  

    print 'validation accuracy: %f' % (val_accuracy, )
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print 'lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy)
    
print 'best validation accuracy achieved during cross-validation: %f' % best_val

(1e-07, 50000.0)
validation accuracy: 0.252000
(1e-07, 100000000.0)
validation accuracy: 0.242000
(5e-07, 50000.0)
validation accuracy: 0.313000
(5e-07, 100000000.0)
validation accuracy: 0.335000
lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.249776 val accuracy: 0.252000
lr 1.000000e-07 reg 1.000000e+08 train accuracy: 0.239776 val accuracy: 0.242000
lr 5.000000e-07 reg 5.000000e+04 train accuracy: 0.316755 val accuracy: 0.313000
lr 5.000000e-07 reg 1.000000e+08 train accuracy: 0.314531 val accuracy: 0.335000
best validation accuracy achieved during cross-validation: 0.335000


In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print 'softmax on raw pixels final test set accuracy: %f' % (test_accuracy, )

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in xrange(10):
  plt.subplot(2, 5, i + 1)
  
  # Rescale the weights to be between 0 and 255
  wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.astype('uint8'))
  plt.axis('off')
  plt.title(classes[i])